# PySpark # 3

##### Índice:

    # Importação bibliotecas / funções
    # Criar Sessão PySpark
    # Criar DF / ler arquivo
    
    # Collect()
    # When() / Otherwise()
    # Union (Concat)
    # Joins
    # Join - Simples
    # Inner Join
    # Left Join
    # Right Join
    # Full Join
    # Semi Join
    # Anti Join

##### Importação bibliotecas / funções

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window # Importando window function

##### Criar / Iniciar Sessão PySpark

In [2]:
spark = (
    SparkSession.builder
    .master('local')
    .appName('PySpark_03')
    .getOrCreate()
)

##### Criar DF / ler arquivo

In [3]:
df = spark.read.csv('wc2018-players.csv', header=True, inferSchema=True)

##### Alterações Aula PySpark 01

In [4]:
df = df.withColumnRenamed('Team', 'Selecao').withColumnRenamed('#', 'Numero').withColumnRenamed('Pos.', 'Posicao')\
.withColumnRenamed('FIFA Popular Name', 'Nome_FIFA').withColumnRenamed('Birth Date', 'Nascimento')\
.withColumnRenamed('Shirt Name', 'Nome Camiseta').withColumnRenamed('Club', 'Time').withColumnRenamed('Height', 'Altura')\
.withColumnRenamed('Weight', 'Peso')

dia = udf(lambda data: data.split('.')[0])
mes = udf(lambda data: data.split('.')[1])
ano = udf(lambda data: data.split('.')[2])

df = df.withColumn('Dia', dia('Nascimento')).withColumn('Mes', mes('Nascimento')).withColumn('Ano', ano('nascimento'))
df = df.withColumn('Data_Nascimento', concat_ws('-', 'Ano', 'Mes', 'Dia').cast(DateType()))

##### Alterações Aula PySpark 02

In [5]:
df2 = df
df = df.drop('Nascimento', 'Nome_FIFA')

##### Distinct()

In [6]:
df.select(col('Selecao')).distinct().show(50)

+--------------+
|       Selecao|
+--------------+
|        Russia|
|       Senegal|
|        Sweden|
|       IR Iran|
|       Germany|
|        France|
|     Argentina|
|       Belgium|
|          Peru|
|       Croatia|
|       Nigeria|
|Korea Republic|
|         Spain|
|       Denmark|
|       Morocco|
|        Panama|
|       Iceland|
|       Uruguay|
|        Mexico|
|       Tunisia|
|  Saudi Arabia|
|   Switzerland|
|        Brazil|
|         Japan|
|       England|
|        Poland|
|      Portugal|
|     Australia|
|    Costa Rica|
|         Egypt|
|        Serbia|
|      Colombia|
+--------------+



##### Collect()

In [7]:
lista = df.select(col('Selecao')).distinct().collect()
lista

[Row(Selecao='Russia'),
 Row(Selecao='Senegal'),
 Row(Selecao='Sweden'),
 Row(Selecao='IR Iran'),
 Row(Selecao='Germany'),
 Row(Selecao='France'),
 Row(Selecao='Argentina'),
 Row(Selecao='Belgium'),
 Row(Selecao='Peru'),
 Row(Selecao='Croatia'),
 Row(Selecao='Nigeria'),
 Row(Selecao='Korea Republic'),
 Row(Selecao='Spain'),
 Row(Selecao='Denmark'),
 Row(Selecao='Morocco'),
 Row(Selecao='Panama'),
 Row(Selecao='Iceland'),
 Row(Selecao='Uruguay'),
 Row(Selecao='Mexico'),
 Row(Selecao='Tunisia'),
 Row(Selecao='Saudi Arabia'),
 Row(Selecao='Switzerland'),
 Row(Selecao='Brazil'),
 Row(Selecao='Japan'),
 Row(Selecao='England'),
 Row(Selecao='Poland'),
 Row(Selecao='Portugal'),
 Row(Selecao='Australia'),
 Row(Selecao='Costa Rica'),
 Row(Selecao='Egypt'),
 Row(Selecao='Serbia'),
 Row(Selecao='Colombia')]

In [8]:
type(lista[0][0])

str

In [9]:
lista[1][0]

'Senegal'

In [10]:
paises =[]

for pais in lista:
    paises.append(pais[0])
    
paises

['Russia',
 'Senegal',
 'Sweden',
 'IR Iran',
 'Germany',
 'France',
 'Argentina',
 'Belgium',
 'Peru',
 'Croatia',
 'Nigeria',
 'Korea Republic',
 'Spain',
 'Denmark',
 'Morocco',
 'Panama',
 'Iceland',
 'Uruguay',
 'Mexico',
 'Tunisia',
 'Saudi Arabia',
 'Switzerland',
 'Brazil',
 'Japan',
 'England',
 'Poland',
 'Portugal',
 'Australia',
 'Costa Rica',
 'Egypt',
 'Serbia',
 'Colombia']

##### When() / Otherwise()

In [26]:
df.withColumn('Coluna_Nova', when(col('Selecao') == "Argentina", 'Argentinos :)').otherwise('NÃO ARGENTINO')).show(50)

+---------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+-------------+
|  Selecao|Numero|Posicao|Nome Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|  Coluna_Nova|
+---------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+-------------+
|Argentina|     3|     DF|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|     1992-08-31|Argentinos :)|
|Argentina|    22|     MF|        PAVÓN|CA Boca Juniors (...|   169|  65| 21| 01|1996|     1996-01-21|Argentinos :)|
|Argentina|    15|     MF|      LANZINI|West Ham United F...|   167|  66| 15| 02|1993|     1993-02-15|Argentinos :)|
|Argentina|    18|     DF|       SALVIO|    SL Benfica (POR)|   167|  69| 13| 07|1990|     1990-07-13|Argentinos :)|
|Argentina|    10|     FW|        MESSI|  FC Barcelona (ESP)|   170|  72| 24| 06|1987|     1987-06-24|Argentinos :)|
|Argentina|     4|     DF|      ANSALDI|     Torino FC (ITA)|   

In [11]:
europa = ['Sweden', 'Germany', 'France', 'Belgium', 'Croatia', 'Spain', 'Denmark', 'Iceland', 'Switzerland', 'England', 'Poland', 'Portugal', 'Serbia']
asia = ['Russia', 'IR Iran', 'Nigeria', 'Korea Republic', 'Saudi Arabia', 'Japan', ]
africa = ['Senegal', 'Morocco', 'Tunisia', 'Egypt']
oceania = ['Australia']
america_norte = ['Panama', 'Mexico', 'Costa Rica']
america_sul = ['Argentina', 'Peru', 'Uruguay', 'Brazil', 'Colombia']

In [12]:
df = df.withColumn('Continente', when(col('Selecao').isin(europa), 'Europa')\
             .when(col('Selecao').isin(asia), 'Ásia')\
             .when(col('Selecao').isin(africa), 'África')\
             .when(col('Selecao').isin(oceania), 'Oceania')\
             .when(col('Selecao').isin(america_norte), 'América do Norte')\
             .when(col('Selecao').isin(america_sul), 'América do Sul')\
             .otherwise('Verificar'))
df.show(5)

+---------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+--------------+
|  Selecao|Numero|Posicao|Nome Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|    Continente|
+---------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+--------------+
|Argentina|     3|     DF|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|     1992-08-31|América do Sul|
|Argentina|    22|     MF|        PAVÓN|CA Boca Juniors (...|   169|  65| 21| 01|1996|     1996-01-21|América do Sul|
|Argentina|    15|     MF|      LANZINI|West Ham United F...|   167|  66| 15| 02|1993|     1993-02-15|América do Sul|
|Argentina|    18|     DF|       SALVIO|    SL Benfica (POR)|   167|  69| 13| 07|1990|     1990-07-13|América do Sul|
|Argentina|    10|     FW|        MESSI|  FC Barcelona (ESP)|   170|  72| 24| 06|1987|     1987-06-24|América do Sul|
+---------+------+-------+-------------+----------------

In [13]:
df.filter('Continente = "Verificar"').show(50)

+-------+------+-------+-------------+----+------+----+---+---+---+---------------+----------+
|Selecao|Numero|Posicao|Nome Camiseta|Time|Altura|Peso|Dia|Mes|Ano|Data_Nascimento|Continente|
+-------+------+-------+-------------+----+------+----+---+---+---+---------------+----------+
+-------+------+-------+-------------+----+------+----+---+---+---+---------------+----------+



##### Union (Concat)

In [14]:
df_america_sul = df.filter('Continente = "América do Sul"')
df_america_sul.select('Selecao').distinct().show()

+---------+
|  Selecao|
+---------+
|Argentina|
|     Peru|
|  Uruguay|
|   Brazil|
| Colombia|
+---------+



In [15]:
df_america_norte = df.filter('Continente = "América do Norte"')
df_america_norte.select('Selecao').distinct().show()

+----------+
|   Selecao|
+----------+
|    Panama|
|    Mexico|
|Costa Rica|
+----------+



In [16]:
df_americas = df_america_sul.union(df_america_norte)

In [17]:
df_americas.show(100)

+---------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+--------------+
|  Selecao|Numero|Posicao|Nome Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|    Continente|
+---------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+--------------+
|Argentina|     3|     DF|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|     1992-08-31|América do Sul|
|Argentina|    22|     MF|        PAVÓN|CA Boca Juniors (...|   169|  65| 21| 01|1996|     1996-01-21|América do Sul|
|Argentina|    15|     MF|      LANZINI|West Ham United F...|   167|  66| 15| 02|1993|     1993-02-15|América do Sul|
|Argentina|    18|     DF|       SALVIO|    SL Benfica (POR)|   167|  69| 13| 07|1990|     1990-07-13|América do Sul|
|Argentina|    10|     FW|        MESSI|  FC Barcelona (ESP)|   170|  72| 24| 06|1987|     1987-06-24|América do Sul|
|Argentina|     4|     DF|      ANSALDI|     Torino FC (

In [18]:
df_americas.select('Selecao').distinct().show()

+----------+
|   Selecao|
+----------+
| Argentina|
|      Peru|
|   Uruguay|
|    Brazil|
|  Colombia|
|    Panama|
|    Mexico|
|Costa Rica|
+----------+



In [19]:
# A QUANTIDADE DE COLUNAS DEVE SER A MESMA

In [20]:
df_america_sul = df_america_sul.drop('Continente')

In [31]:
df_america_norte.show(4)

+----------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+----------+
|   Selecao|Numero|Posicao|Nome Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|Continente|
+----------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+----------+
|Costa Rica|     7|     MF|   C. BOLAÑOS|Deportivo Sapriss...|   178|  67| 17| 05|1984|     1984-05-17|       100|
|Costa Rica|    16|     DF|    C. GAMBOA|     Celtic FC (SCO)|   175|  67| 24| 10|1989|     1989-10-24|       100|
|Costa Rica|    17|     MF|    Y. TEJEDA|FC Lausanne-Sport...|   179|  67| 17| 03|1992|     1992-03-17|       100|
|Costa Rica|     4|     DF|     I. SMITH|IFK Norrkoping FK...|   177|  68| 06| 03|1998|     1998-03-06|       100|
+----------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+----------+
only showing top 4 rows



In [32]:
df_americas = df_america_sul.union(df_america_norte)

AnalysisException: [NUM_COLUMNS_MISMATCH] UNION can only be performed on inputs with the same number of columns, but the first input has 11 columns and the second input has 12 columns.;
'Union false, false
:- Project [Selecao#35, Numero#46, Posicao#56, Nome Camiseta#86, Time#96, Altura#106, Peso#116, Dia#127, Mes#139, Ano#152, Data_Nascimento#165]
:  +- Filter (Continente#199 = América do Sul)
:     +- Project [Selecao#35, Numero#46, Posicao#56, Nome Camiseta#86, Time#96, Altura#106, Peso#116, Dia#127, Mes#139, Ano#152, Data_Nascimento#165, CASE WHEN Selecao#35 IN (Sweden,Germany,France,Belgium,Croatia,Spain,Denmark,Iceland,Switzerland,England,Poland,Portugal,Serbia) THEN Europa WHEN Selecao#35 IN (Russia,IR Iran,Nigeria,Korea Republic,Saudi Arabia,Japan) THEN Ásia WHEN Selecao#35 IN (Senegal,Morocco,Tunisia,Egypt) THEN África WHEN Selecao#35 IN (Australia) THEN Oceania WHEN Selecao#35 IN (Panama,Mexico,Costa Rica) THEN América do Norte WHEN Selecao#35 IN (Argentina,Peru,Uruguay,Brazil

In [ ]:
# COLUNAS COM TIPOS OU NOMES DIFERENTES TAMBÉM CONCATENAM

# OBS: Cuidado para não concatenar colunas em posições diferentes

In [33]:
df_america_norte = df_america_norte.withColumn('Continente', lit(100))
df_america_norte.printSchema()

root
 |-- Selecao: string (nullable = true)
 |-- Numero: integer (nullable = true)
 |-- Posicao: string (nullable = true)
 |-- Nome Camiseta: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Altura: integer (nullable = true)
 |-- Peso: integer (nullable = true)
 |-- Dia: string (nullable = true)
 |-- Mes: string (nullable = true)
 |-- Ano: string (nullable = true)
 |-- Data_Nascimento: date (nullable = true)
 |-- Continente: integer (nullable = false)



In [34]:
df_america_norte.show(6)

+----------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+----------+
|   Selecao|Numero|Posicao|Nome Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|Continente|
+----------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+----------+
|Costa Rica|     7|     MF|   C. BOLAÑOS|Deportivo Sapriss...|   178|  67| 17| 05|1984|     1984-05-17|       100|
|Costa Rica|    16|     DF|    C. GAMBOA|     Celtic FC (SCO)|   175|  67| 24| 10|1989|     1989-10-24|       100|
|Costa Rica|    17|     MF|    Y. TEJEDA|FC Lausanne-Sport...|   179|  67| 17| 03|1992|     1992-03-17|       100|
|Costa Rica|     4|     DF|     I. SMITH|IFK Norrkoping FK...|   177|  68| 06| 03|1998|     1998-03-06|       100|
|Costa Rica|     8|     DF|    B. OVIEDO|Sunderland AFC (ENG)|   172|  70| 18| 02|1990|     1990-02-18|       100|
|Costa Rica|    13|     MF|   R. WALLACE|New York City FC ...|   180|  70| 17| 0

In [35]:
df_americas = df_america_norte.union(df_america_sul)
df_americas.show(200)

AnalysisException: [NUM_COLUMNS_MISMATCH] UNION can only be performed on inputs with the same number of columns, but the first input has 12 columns and the second input has 11 columns.;
'Union false, false
:- Project [Selecao#35, Numero#46, Posicao#56, Nome Camiseta#86, Time#96, Altura#106, Peso#116, Dia#127, Mes#139, Ano#152, Data_Nascimento#165, 100 AS Continente#914]
:  +- Project [Selecao#35, Numero#46, Posicao#56, Nome Camiseta#86, Time#96, Altura#106, Peso#116, Dia#127, Mes#139, Ano#152, Data_Nascimento#165, 100 AS Continente#564]
:     +- Filter (Continente#199 = América do Norte)
:        +- Project [Selecao#35, Numero#46, Posicao#56, Nome Camiseta#86, Time#96, Altura#106, Peso#116, Dia#127, Mes#139, Ano#152, Data_Nascimento#165, CASE WHEN Selecao#35 IN (Sweden,Germany,France,Belgium,Croatia,Spain,Denmark,Iceland,Switzerland,England,Poland,Portugal,Serbia) THEN Europa WHEN Selecao#35 IN (Russia,IR Iran,Nigeria,Korea Republic,Saudi Arabia,Japan) THEN Ásia WHEN Selecao#35 IN (Sen

In [27]:
df_america_sul = df_america_sul.withColumnRenamed('Continente', 'NOVA COLUNA DOIDA')

In [28]:
df_america_norte.show(5)

+----------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+----------+
|   Selecao|Numero|Posicao|Nome Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|Continente|
+----------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+----------+
|Costa Rica|     7|     MF|   C. BOLAÑOS|Deportivo Sapriss...|   178|  67| 17| 05|1984|     1984-05-17|       100|
|Costa Rica|    16|     DF|    C. GAMBOA|     Celtic FC (SCO)|   175|  67| 24| 10|1989|     1989-10-24|       100|
|Costa Rica|    17|     MF|    Y. TEJEDA|FC Lausanne-Sport...|   179|  67| 17| 03|1992|     1992-03-17|       100|
|Costa Rica|     4|     DF|     I. SMITH|IFK Norrkoping FK...|   177|  68| 06| 03|1998|     1998-03-06|       100|
|Costa Rica|     8|     DF|    B. OVIEDO|Sunderland AFC (ENG)|   172|  70| 18| 02|1990|     1990-02-18|       100|
+----------+------+-------+-------------+--------------------+------+----+---+--

In [29]:
df_americas = df_america_sul.union(df_america_norte)
df_americas.show(200)

AnalysisException: [NUM_COLUMNS_MISMATCH] UNION can only be performed on inputs with the same number of columns, but the first input has 11 columns and the second input has 12 columns.;
'Union false, false
:- Project [Selecao#35, Numero#46, Posicao#56, Nome Camiseta#86, Time#96, Altura#106, Peso#116, Dia#127, Mes#139, Ano#152, Data_Nascimento#165]
:  +- Filter (Continente#199 = América do Sul)
:     +- Project [Selecao#35, Numero#46, Posicao#56, Nome Camiseta#86, Time#96, Altura#106, Peso#116, Dia#127, Mes#139, Ano#152, Data_Nascimento#165, CASE WHEN Selecao#35 IN (Sweden,Germany,France,Belgium,Croatia,Spain,Denmark,Iceland,Switzerland,England,Poland,Portugal,Serbia) THEN Europa WHEN Selecao#35 IN (Russia,IR Iran,Nigeria,Korea Republic,Saudi Arabia,Japan) THEN Ásia WHEN Selecao#35 IN (Senegal,Morocco,Tunisia,Egypt) THEN África WHEN Selecao#35 IN (Australia) THEN Oceania WHEN Selecao#35 IN (Panama,Mexico,Costa Rica) THEN América do Norte WHEN Selecao#35 IN (Argentina,Peru,Uruguay,Brazil

##### Joins

In [36]:
df.show(5)

+---------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+--------------+
|  Selecao|Numero|Posicao|Nome Camiseta|                Time|Altura|Peso|Dia|Mes| Ano|Data_Nascimento|    Continente|
+---------+------+-------+-------------+--------------------+------+----+---+---+----+---------------+--------------+
|Argentina|     3|     DF|   TAGLIAFICO|      AFC Ajax (NED)|   169|  65| 31| 08|1992|     1992-08-31|América do Sul|
|Argentina|    22|     MF|        PAVÓN|CA Boca Juniors (...|   169|  65| 21| 01|1996|     1996-01-21|América do Sul|
|Argentina|    15|     MF|      LANZINI|West Ham United F...|   167|  66| 15| 02|1993|     1993-02-15|América do Sul|
|Argentina|    18|     DF|       SALVIO|    SL Benfica (POR)|   167|  69| 13| 07|1990|     1990-07-13|América do Sul|
|Argentina|    10|     FW|        MESSI|  FC Barcelona (ESP)|   170|  72| 24| 06|1987|     1987-06-24|América do Sul|
+---------+------+-------+-------------+----------------

In [37]:
arg = df.filter('Selecao = "Argentina"')
bra = df.filter('Selecao = "Brazil"')

In [38]:
arg = arg.drop('Time', 'Dia', 'Mes', 'Ano', 'Continente', 'Peso', 'Data_Nascimento')
bra = bra.drop('Time', 'Dia', 'Mes', 'Ano', 'Continente', 'Peso', 'Data_Nascimento')

In [39]:
arg.show(5)

+---------+------+-------+-------------+------+
|  Selecao|Numero|Posicao|Nome Camiseta|Altura|
+---------+------+-------+-------------+------+
|Argentina|     3|     DF|   TAGLIAFICO|   169|
|Argentina|    22|     MF|        PAVÓN|   169|
|Argentina|    15|     MF|      LANZINI|   167|
|Argentina|    18|     DF|       SALVIO|   167|
|Argentina|    10|     FW|        MESSI|   170|
+---------+------+-------+-------------+------+
only showing top 5 rows



In [40]:
bra.show(50)

+-------+------+-------+-------------+------+
|Selecao|Numero|Posicao|Nome Camiseta|Altura|
+-------+------+-------+-------------+------+
| Brazil|    18|     MF|         FRED|   169|
| Brazil|    21|     FW|       TAISON|   172|
| Brazil|    17|     MF|  FERNANDINHO|   179|
| Brazil|    22|     DF|       FAGNER|   168|
| Brazil|    10|     FW|    NEYMAR JR|   175|
| Brazil|    11|     MF|  P. COUTINHO|   172|
| Brazil|     7|     FW|     D. COSTA|   182|
| Brazil|     6|     DF|  FILIPE LUIS|   182|
| Brazil|     9|     FW|     G. JESUS|   175|
| Brazil|    13|     DF|   MARQUINHOS|   183|
| Brazil|    20|     FW|      FIRMINO|   181|
| Brazil|    19|     MF|      WILLIAN|   175|
| Brazil|     3|     DF|      MIRANDA|   186|
| Brazil|    14|     DF|       DANILO|   184|
| Brazil|     2|     DF|     T. SILVA|   183|
| Brazil|    12|     DF|      MARCELO|   174|
| Brazil|    15|     MF|     PAULINHO|   181|
| Brazil|     4|     DF|      GEROMEL|   190|
| Brazil|     5|     MF|     CASEM

In [41]:
arg.count()

23

##### Join - Simples

In [ ]:
# Dados dos joins 100% correspondentes

In [42]:
dfnovo = arg.join(bra, arg.Numero == bra.Numero)
dfnovo.show(25)

+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|  Selecao|Numero|Posicao|Nome Camiseta|Altura|Selecao|Numero|Posicao|Nome Camiseta|Altura|
+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|Argentina|     3|     DF|   TAGLIAFICO|   169| Brazil|     3|     DF|      MIRANDA|   186|
|Argentina|    22|     MF|        PAVÓN|   169| Brazil|    22|     DF|       FAGNER|   168|
|Argentina|    15|     MF|      LANZINI|   167| Brazil|    15|     MF|     PAULINHO|   181|
|Argentina|    18|     DF|       SALVIO|   167| Brazil|    18|     MF|         FRED|   169|
|Argentina|    10|     FW|        MESSI|   170| Brazil|    10|     FW|    NEYMAR JR|   175|
|Argentina|     4|     DF|      ANSALDI|   181| Brazil|     4|     DF|      GEROMEL|   190|
|Argentina|     5|     MF|       BIGLIA|   175| Brazil|     5|     MF|     CASEMIRO|   185|
|Argentina|     7|     MF|       BANEGA|   175| Brazil|     7|     FW|     D. CO

In [43]:
arg = arg.withColumn('Numero', col('Numero') + 1)
arg.show(23)

+---------+------+-------+-------------+------+
|  Selecao|Numero|Posicao|Nome Camiseta|Altura|
+---------+------+-------+-------------+------+
|Argentina|     4|     DF|   TAGLIAFICO|   169|
|Argentina|    23|     MF|        PAVÓN|   169|
|Argentina|    16|     MF|      LANZINI|   167|
|Argentina|    19|     DF|       SALVIO|   167|
|Argentina|    11|     FW|        MESSI|   170|
|Argentina|     5|     DF|      ANSALDI|   181|
|Argentina|     6|     MF|       BIGLIA|   175|
|Argentina|     8|     MF|       BANEGA|   175|
|Argentina|    15|     DF|   MASCHERANO|   174|
|Argentina|    22|     FW|       DYBALA|   177|
|Argentina|    20|     FW|       AGÜERO|   172|
|Argentina|    10|     FW|      HIGUAÍN|   184|
|Argentina|    12|     MF|     DI MARÍA|   178|
|Argentina|    21|     MF|     LO CELSO|   177|
|Argentina|    14|     MF|         MEZA|   180|
|Argentina|     9|     DF|        ACUÑA|   172|
|Argentina|    24|     GK|    CABALLERO|   186|
|Argentina|     3|     DF|      MERCADO|

##### Inner Join

In [ ]:
# Mostra apenas os dados que têm correspondências

In [44]:
dfnovo = arg.join(bra, arg['Numero'] == bra['Numero'], 'inner')
dfnovo.show(40)

+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|  Selecao|Numero|Posicao|Nome Camiseta|Altura|Selecao|Numero|Posicao|Nome Camiseta|Altura|
+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|Argentina|     4|     DF|   TAGLIAFICO|   169| Brazil|     4|     DF|      GEROMEL|   190|
|Argentina|    23|     MF|        PAVÓN|   169| Brazil|    23|     GK|      EDERSON|   188|
|Argentina|    16|     MF|      LANZINI|   167| Brazil|    16|     GK|       CASSIO|   195|
|Argentina|    19|     DF|       SALVIO|   167| Brazil|    19|     MF|      WILLIAN|   175|
|Argentina|    11|     FW|        MESSI|   170| Brazil|    11|     MF|  P. COUTINHO|   172|
|Argentina|     5|     DF|      ANSALDI|   181| Brazil|     5|     MF|     CASEMIRO|   185|
|Argentina|     6|     MF|       BIGLIA|   175| Brazil|     6|     DF|  FILIPE LUIS|   182|
|Argentina|     8|     MF|       BANEGA|   175| Brazil|     8|     MF|   R. AUGU

##### Left Join

In [ ]:
# Mostra todos os dados do DF do lado ESQUERDO
# (valores do lado dir que não tiverem correspondências, serão mostrados como nulos)

In [45]:
dfnovo = arg.join(bra, arg['Numero'] == bra['Numero'], 'left')
dfnovo.show(40)

+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|  Selecao|Numero|Posicao|Nome Camiseta|Altura|Selecao|Numero|Posicao|Nome Camiseta|Altura|
+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|Argentina|     4|     DF|   TAGLIAFICO|   169| Brazil|     4|     DF|      GEROMEL|   190|
|Argentina|    23|     MF|        PAVÓN|   169| Brazil|    23|     GK|      EDERSON|   188|
|Argentina|    16|     MF|      LANZINI|   167| Brazil|    16|     GK|       CASSIO|   195|
|Argentina|    19|     DF|       SALVIO|   167| Brazil|    19|     MF|      WILLIAN|   175|
|Argentina|    11|     FW|        MESSI|   170| Brazil|    11|     MF|  P. COUTINHO|   172|
|Argentina|     5|     DF|      ANSALDI|   181| Brazil|     5|     MF|     CASEMIRO|   185|
|Argentina|     6|     MF|       BIGLIA|   175| Brazil|     6|     DF|  FILIPE LUIS|   182|
|Argentina|     8|     MF|       BANEGA|   175| Brazil|     8|     MF|   R. AUGU

##### Right Join

In [ ]:
# Mostra todos os dados do DF do lado DIREITO
# (valores do lado esquerdo que não tiverem correspondências, serão mostrados como nulos)

In [46]:
dfnovo = arg.join(bra, arg['Numero'] == bra['Numero'], 'right')
dfnovo.show(40)

+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|  Selecao|Numero|Posicao|Nome Camiseta|Altura|Selecao|Numero|Posicao|Nome Camiseta|Altura|
+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|Argentina|    18|     DF|     OTAMENDI|   181| Brazil|    18|     MF|         FRED|   169|
|Argentina|    21|     MF|     LO CELSO|   177| Brazil|    21|     FW|       TAISON|   172|
|Argentina|    17|     DF|         ROJO|   189| Brazil|    17|     MF|  FERNANDINHO|   179|
|Argentina|    22|     FW|       DYBALA|   177| Brazil|    22|     DF|       FAGNER|   168|
|Argentina|    10|     FW|      HIGUAÍN|   184| Brazil|    10|     FW|    NEYMAR JR|   175|
|Argentina|    11|     FW|        MESSI|   170| Brazil|    11|     MF|  P. COUTINHO|   172|
|Argentina|     7|     DF|        FAZIO|   199| Brazil|     7|     FW|     D. COSTA|   182|
|Argentina|     6|     MF|       BIGLIA|   175| Brazil|     6|     DF|  FILIPE L

##### Full Join

In [ ]:
# Será mostrado todas as linhas
# (valores que não corresponderem, serão mostrado como nulos em ambos os lados)

In [47]:
dfnovo = arg.join(bra, arg['Numero'] == bra['Numero'], 'full')
dfnovo.show(40)

+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|  Selecao|Numero|Posicao|Nome Camiseta|Altura|Selecao|Numero|Posicao|Nome Camiseta|Altura|
+---------+------+-------+-------------+------+-------+------+-------+-------------+------+
|     NULL|  NULL|   NULL|         NULL|  NULL| Brazil|     1|     GK|    A. BECKER|   193|
|Argentina|     2|     GK|       GUZMÁN|   192| Brazil|     2|     DF|     T. SILVA|   183|
|Argentina|     3|     DF|      MERCADO|   181| Brazil|     3|     DF|      MIRANDA|   186|
|Argentina|     4|     DF|   TAGLIAFICO|   169| Brazil|     4|     DF|      GEROMEL|   190|
|Argentina|     5|     DF|      ANSALDI|   181| Brazil|     5|     MF|     CASEMIRO|   185|
|Argentina|     6|     MF|       BIGLIA|   175| Brazil|     6|     DF|  FILIPE LUIS|   182|
|Argentina|     7|     DF|        FAZIO|   199| Brazil|     7|     FW|     D. COSTA|   182|
|Argentina|     8|     MF|       BANEGA|   175| Brazil|     8|     MF|   R. AUGU

##### Semi Join

In [ ]:
# Similar ao INNER JOIN, porém apenas os dados do DataFrame esquerdo é mostrado

In [48]:
dfnovo = bra.join(arg, arg['Numero'] == bra['Numero'], 'semi')
dfnovo.show(40)

+-------+------+-------+-------------+------+
|Selecao|Numero|Posicao|Nome Camiseta|Altura|
+-------+------+-------+-------------+------+
| Brazil|    18|     MF|         FRED|   169|
| Brazil|    21|     FW|       TAISON|   172|
| Brazil|    17|     MF|  FERNANDINHO|   179|
| Brazil|    22|     DF|       FAGNER|   168|
| Brazil|    10|     FW|    NEYMAR JR|   175|
| Brazil|    11|     MF|  P. COUTINHO|   172|
| Brazil|     7|     FW|     D. COSTA|   182|
| Brazil|     6|     DF|  FILIPE LUIS|   182|
| Brazil|     9|     FW|     G. JESUS|   175|
| Brazil|    13|     DF|   MARQUINHOS|   183|
| Brazil|    20|     FW|      FIRMINO|   181|
| Brazil|    19|     MF|      WILLIAN|   175|
| Brazil|     3|     DF|      MIRANDA|   186|
| Brazil|    14|     DF|       DANILO|   184|
| Brazil|     2|     DF|     T. SILVA|   183|
| Brazil|    12|     DF|      MARCELO|   174|
| Brazil|    15|     MF|     PAULINHO|   181|
| Brazil|     4|     DF|      GEROMEL|   190|
| Brazil|     5|     MF|     CASEM

##### Anti Join

In [ ]:
# Mostra os dados do DF do lado esquerdo que NÃO possuem correspondências

In [49]:
dfnovo = arg.join(bra, arg['Numero'] == bra['Numero'], 'anti')
dfnovo.show(40)

+---------+------+-------+-------------+------+
|  Selecao|Numero|Posicao|Nome Camiseta|Altura|
+---------+------+-------+-------------+------+
|Argentina|    24|     GK|    CABALLERO|   186|
+---------+------+-------+-------------+------+



----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 51647)
Traceback (most recent call last):
  File "c:\Users\Luan Lima\AppData\Local\Programs\Python\Python310\lib\socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "c:\Users\Luan Lima\AppData\Local\Programs\Python\Python310\lib\socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "c:\Users\Luan Lima\AppData\Local\Programs\Python\Python310\lib\socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "c:\Users\Luan Lima\AppData\Local\Programs\Python\Python310\lib\socketserver.py", line 747, in __init__
    self.handle()
  File "c:\Users\Luan Lima\AppData\Local\Programs\Python\Python310\lib\site-packages\pyspark\accumulators.py", line 295, in handle
    poll(accum_updates)
  File "c:\Users\Luan Lima\AppData\Local\Progr